Afegir Class, SubClass, blabla

In [2]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#### Tornem a carregar el primer nivell

In [3]:
odor = pd.read_json('Data/odor_prop.json', lines=False, orient='records')
odor=odor[odor['level']=='first']
odor.reset_index(drop=True, inplace=True)
mol=odor['cid'].value_counts()
mols=mol.index[mol>1]
groups=[]
for i in range(len(odor)):
    for mol in mols:
        if odor['cid'][i] == mol:
            g=np.max(odor[odor['cid']==mol]['label'])
            groups.append([mol, g])
            ls_drop=[]
for mol in groups:
    for i in range(len(odor)):
        if (odor['cid'][i] == mol[0]):
            if odor['label'][i]!=mol[1]:
                ls_drop.append(i)
odor=odor.drop(ls_drop)
odor.reset_index(drop=True, inplace=True)
odor = odor.sort_values(by='cid')
odor.reset_index(inplace=True)
ls_repeat=[]
for i in range(len(odor)-1):
    if (odor['cid'][i]==odor['cid'][i+1]):
        ls_repeat.append(i)
odor=odor.drop(ls_repeat)

#### Canviem les variables categòriques d'str a int:

In [4]:
len(odor['Subclass'].unique()), len(odor['Superclass'].unique()), len(odor['Class'].unique()), len(odor['Direct Parent'].unique())

(69, 10, 65, 135)

In [5]:
odor.replace(to_replace=odor['Subclass'].unique(), 
                   value=np.arange(0,69), inplace=True)

In [6]:
odor.replace(to_replace=odor['Superclass'].unique(), 
                   value=np.arange(0,10), inplace=True)
odor.replace(to_replace=odor['Class'].unique(), 
                   value=np.arange(0,65), inplace=True)
odor.replace(to_replace=odor['Direct Parent'].unique(), 
                   value=np.arange(0,113), inplace=True)

In [7]:
odor.columns

Index(['index', 'name', 'level', 'cid', 'label', 'smiles', 'TPSA', 'MolLogP',
       'MolWt', 'ExactMolWt', 'MolMR', 'NumHAcceptors', 'NumHDonors',
       'NumRotatableBonds', 'NumAromaticRings', 'NumSaturatedRings',
       'NumAliphaticRings', 'NumAromaticHeterocycles',
       'NumSaturatedHeterocycles', 'NumAliphaticHeterocycles',
       'NumAromaticCarbocycles', 'NumSaturatedCarbocycles',
       'NumAliphaticCarbocycles', 'NumHeteroatoms', 'NumValenceElectrons',
       'RingCount', 'FractionCSP3', 'LabuteASA', 'BalabanJ', 'BertzCT', 'Ipc',
       'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Chi0', 'Chi1', 'Chi0n',
       'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v',
       'Chi4v', 'HeavyAtom', 'Gasteiger', 'Eccentricity',
       'InertialShapeFactor', 'RadiusOfGyration', 'Asphericity',
       'SpherocityIndex', 'PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2',
       'Molecule Name', 'InChIKey', 'SMILES', 'Kingdom', 'Superclass', 'Class',
       'Subclass', 'Inte

In [8]:
ind_X=['TPSA', 'MolLogP',
       'MolWt', 'ExactMolWt', 'MolMR', 'NumHAcceptors', 'NumHDonors',
       'NumRotatableBonds', 'NumAromaticRings', 'NumSaturatedRings',
       'NumAliphaticRings', 'NumAromaticHeterocycles',
       'NumSaturatedHeterocycles', 'NumAliphaticHeterocycles',
       'NumAromaticCarbocycles', 'NumSaturatedCarbocycles',
       'NumAliphaticCarbocycles', 'NumHeteroatoms', 'NumValenceElectrons',
       'RingCount', 'FractionCSP3', 'LabuteASA', 'BalabanJ', 'BertzCT', 'Ipc',
       'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Chi0', 'Chi1', 'Chi0n',
       'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v',
       'Chi4v', 'HeavyAtom', 'Gasteiger', 'Eccentricity',
       'InertialShapeFactor', 'RadiusOfGyration', 'Asphericity',
       'SpherocityIndex', 'PMI1', 'PMI2', 'PMI3', 'NPR1', 'NPR2',
       'Superclass', 'Class', 'Subclass', 'Direct Parent'
      ]

In [9]:
ind_X=odor.columns[6:59]
X = odor[ind_X]
y = odor['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42) 

In [14]:
classifiers = [
    KNeighborsClassifier(3),
    KNeighborsClassifier(5),
    KNeighborsClassifier(8),
    KNeighborsClassifier(12),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LinearDiscriminantAnalysis(),
    make_pipeline(StandardScaler(), SVC(gamma='auto')),
    svm.SVC()
    ]

# Logging for Visual Comparison


for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
      
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 34.2270%
KNeighborsClassifier
****Results****
Accuracy: 36.4611%
KNeighborsClassifier
****Results****
Accuracy: 38.5165%
KNeighborsClassifier
****Results****
Accuracy: 37.4441%
DecisionTreeClassifier
****Results****
Accuracy: 34.3164%
RandomForestClassifier
****Results****
Accuracy: 45.9339%
LinearDiscriminantAnalysis
****Results****
Accuracy: 47.2744%
Pipeline
****Results****
Accuracy: 45.1296%
SVC
****Results****
Accuracy: 32.7971%
